# Test SoilGrids v2 access through Earth Engine

https://git.wur.nl/isric/soilgrids/soilgrids.notebooks/-/blob/master/markdown/access_on_gee.md

#### Run zonal stats for SIBBORK sites

In [1]:
import os, sys
import ee
import geopandas as gpd
import pandas as pd

/tmp/ipykernel_702/1464670107.py:3: DeprecationWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In the next release, GeoPandas will switch to using Shapely by default, even if PyGEOS is installed. If you only have PyGEOS installed to get speed-ups, this switch should be smooth. However, if you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


In [ ]:
!pip install geemap

In [4]:
import geemap

In [122]:
Map = geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [121]:
# # Add Earth Engine dataset
# SOIL_VAR_NAME = "cec"
# data = ee.Image(f"projects/soilgrids-isric/{SOIL_VAR_NAME}_mean")

# BANDNAME = f"{SOIL_VAR_NAME}_0-5cm_mean"

# # Set visualization parameters.
# data_vis = {
#     "bands":[BANDNAME],
#     'min': 0,
#     'max': 800,
#     'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5'],
# }

# # Add Earth Engine DEM to map
# Map.addLayer(data, data_vis, f'SoilGrid v2: {BANDNAME}')

### Read the input zones file

In [123]:
input_zones_fn = "/projects/my-public-bucket/footprints_LCunmasked.gpkg"

footprints_sibbork_gdf = gpd.read_file(input_zones_fn)

# convert to FeatureCollection using one line of code
footprints_sibbork_fc = geemap.geopandas_to_ee(footprints_sibbork_gdf)

Map.addLayer(footprints_sibbork_fc, {"color":"red"}, 'SIBBORK sites TTE')

### Set some zonal stats arguments

In [ ]:
STATS_TYPE = 'MEDIAN'
SCALE = 250
# out_dir = '/projects/my-public-bucket/test_geemap_zonal'
# if not os.path.exists(out_dir):
#     os.makedirs(out_dir)

### Run zonal stats across all layers of SoilGrids V2 and append to the input geodataframe

In [129]:
SOILGRIDS_LAYER_LIST = ["bdod","cec","cfvo","clay","nitrogen","ocd","ocs","phh2o","sand","silt","soc"]
df_list = []

for SOILGRID in SOILGRIDS_LAYER_LIST:
    
    data = ee.Image(f"projects/soilgrids-isric/{SOILGRID}_mean")

    out_stats_fn = os.path.join(out_dir, f'zonal_stats_{STATS_TYPE}_{SCALE}_{SOILGRID}' + '.csv')

    # Allowed output formats: csv, shp, json, kml, kmz
    # Allowed statistics type: MEAN, MAXIMUM, MINIMUM, MEDIAN, STD, MIN_MAX, VARIANCE, SUM
    df = geemap.zonal_statistics(data, footprints_sibbork_fc, out_stats_fn, statistics_type=STATS_TYPE, scale=SCALE, return_fc=True)
    df = geemap.ee_to_pandas(df)
    df_list.append(df.drop(columns=COLS_LIST[0:-1]))
    
df = pd.concat(df_list, axis = 1).T.drop_duplicates().T

Computing statistics ...
Computing statistics ...
Computing statistics ...
Computing statistics ...
Computing statistics ...
Computing statistics ...
Computing statistics ...
Computing statistics ...
Computing statistics ...
Computing statistics ...
Computing statistics ...


### Save to gpkg and csv

In [134]:
footprints_sibbork_gdf_soils = pd.concat([gdf, df], axis=1)
footprints_sibbork_gdf_soils.to_file(os.path.splitext(input_zones_fn)[0] + f'_zonal_stats_{STATS_TYPE}_{SCALE}' + '.gpkg')
footprints_sibbork_gdf_soils.to_csv(os.path.splitext(input_zones_fn)[0] + f'_zonal_stats_{STATS_TYPE}_{SCALE}' + '.csv')
footprints_sibbork_gdf_soils.head()

,name,file,area_km2,area_ha,site_cnt_h5,site_sim_status,site_cnt_stemmap_yrs,geometry,bdod_0-5cm_mean,bdod_5-15cm_mean,...,silt_15-30cm_mean,silt_30-60cm_mean,silt_60-100cm_mean,silt_100-200cm_mean,soc_0-5cm_mean,soc_5-15cm_mean,soc_15-30cm_mean,soc_30-60cm_mean,soc_60-100cm_mean,soc_100-200cm_mean
0,Creamer_823_1_49,arcticDEM_Creamer_823_1_49_10m_mf300_LCunmaske...,9.0,900.0,0,incomplete,0.0,"POLYGON ((-2699360.000 607680.000, -2699360.00...",55.421099,70.692421,...,689.165080,684.089266,669.271912,677.688702,2114.798192,1359.857235,583.380553,289.045205,260.226394,257.604548
1,TL_MM27_T1_1_11,arcticDEM_TL_MM27_T1_1_11_10m_mf300_LCunmasked...,9.0,900.0,3,complete,93.0,"POLYGON ((-2385410.000 1429930.000, -2385410.0...",64.680808,83.423659,...,460.851899,440.465877,433.973202,391.566854,2915.137860,2142.926030,1302.106539,963.524519,639.078428,618.847006
2,ambler,arcticDEM_ambler_10m_mf300_LCunmasked.tif,9.0,900.0,3,complete,93.0,"POLYGON ((-2292720.000 915650.000, -2292720.00...",57.420586,81.637723,...,403.675696,395.056087,353.153423,328.230767,2574.497342,1619.058397,830.545633,566.860911,502.936610,519.919014
3,brooks01,arcticDEM_brooks01_10m_mf300_LCunmasked.tif,9.0,900.0,3,complete,93.0,"POLYGON ((-2385410.000 637580.000, -2385410.00...",48.406301,74.048485,...,439.235207,412.689173,369.947119,331.084748,2762.913914,1907.273818,1200.982772,671.229759,607.671574,602.170030
4,brooks02,arcticDEM_brooks02_10m_mf300_LCunmasked.tif,9.0,900.0,3,complete,93.0,"POLYGON ((-2388400.000 640570.000, -2388400.00...",50.818985,74.706796,...,430.547498,418.552321,361.440402,324.088248,2648.198053,1944.254302,1270.870353,879.338660,783.145375,780.942497
